# Variable scoping

Armed with the knowledge we have just acquired, let us consider the following program:

```
k := 0
x := 0
incr_x := <k => x := x + k>
incr_x(1)
incr_x(3)
...
```
| Program | State|
|:-------:|:---------:|
| `< k := 0 ; ... >` | $\{\}$|
| `< x := 0 ; ... >` | $\{ k := 0 \}$|
| `< incr_x := <k => x := x + k> ; ... >` | $\{ k := 0 , x:=0 \}$|
| `< incr_x(1) ; ... >` | $\{ k := 0 , x:=0 , incr\_x := <k \Rightarrow x := x + k> \}$|
| `< k := 1 ; x := x + k ; ... >` | $ ... $|
| `< x := x + k ; ... >` | $\{ k := 1 , x:=0 , incr\_x := <k \Rightarrow x := x + k> \}$|
| `< x := 0 + k ; ... >` | $ ... $|
| `< x := 0 + 1 ; ... >` | $ ... $|
| `< x := 1 ; ... >` | $ ... $|
| `< incr_x(3) ; ... >` | $ ... $|
| `< k := 3 ; x := x + k ; ... >` | $ ... $|
| `< x := x + k ; ... >` | $\{ k := 3 , x:=1 , incr\_x := <k \Rightarrow x := x + k> \}$|
| `< x := 1 + k ; ... >` | $ ... $|
| `< x := 1 + 3 ; ... >` | $ ... $|
| `< x := 4 ; ... >` | $ ... $|
| `< ... >` | $\{ k := 3 , x:=4 , incr\_x := <k \Rightarrow x := x + k> \}$|

If we follow the semantics we have given in the previous lectures, then when we invoke `incr_x`, we do not just *declare* a new variable `k` in the state: we *overwrite* it. Overwriting a variable is a dangerous business. First of all, (over)writing a variable is referred to as a *destructive update*. It is called destructive because the value that was contained in the state is destroyed (we cannot read it anymore) in favour of the new value. Calling a function named `incr_x` not-so-subtly implies that the function will perform one task and one task only: increment variable `x`. Unfortunately, the function also has an unseen side-effect: it also resets the value of `k` to whatever we increment `x` by. This is very unfortunate, and to obviate this we might want to call the function `incr_x_and_reset_k`, but this would not really be the proper solution: it is a cumbersome name, and it reduces the usefulness of our function. An abstraction, such as a function, is considered to be *leaky* when it produces a change in some constructs which are not directly related to the main task of the function, or when it forces us to consider implementation details (such as the fact that `k` is reset). If you remember, the reason why we introduced functions in the first place was to ensure that we could only focus on what matters (such as incrementing `x`) while not having to keep in mind all the details needed to _make it happen_ (such as what happens with local data storage such as variable `k`), so this is actually quite a disappointing failure.

## Scope
To obviate this issue, we might observe that even though the code in the sample above does manipulate variable `k`, this variable acquires three very separate meanings during execution of the program: 
- in the beginning, the variable is just a global variable;
- during the first call to `incr_x`, `k` stores value $1$ (how much we want to increase `x`);
- during the second call to `incr_x`, `k` stores value $3$ (how much we want to increase `x`).

The reason why we consider these meanings to be distinct is that the second value that `k` acquires, $1$, is only useful and meaningful **inside the lambda**, but it should not interfere with (nor destroy) the previous value of `k`, which should remain (or return to) $0$ after the lambda is done computing. We might say that even though the name of the variable is the same, the variable itself belongs to a different *context*, and thus the name and the context define **three separate variables**: one global, with value $0$, one local to the first call to `incr_x` and with value $1$, and a last local one for the second call to `incr_x` and with value $3$.

This notion of the context inside which a variable lives is called _scope_. Variables can either live in the *global* scope (the one we have used so far) or in the *local* scope defined by a function, and then being erased when the code of the function is done. If a local variable has the same name as a global variable, then it takes precedence. This means that the local `k` would be used for the function `incr_x`, whereas the global `k` would be left untouched (and temporarily ignored during the computation of the function `incr_x`).

In order to implement scope, we need to adjust our semantics for function invocation and variable lookup. We want to distinguish, when invoking a function, that its code is being run in a new and separate scope, and to distinguish scopes in the state.

Scopes in the state are stored in the so-called _call stack_: a stack which contains all local variables of functions being active at the current time (yes, there may be more than one: more on this later). When a function is invoked, its parameters are *instantiated with* (= associated to the values of) the given arguments at the top of the stack. When the function is done, we remove the whole top of the stack in bulk, thereby erasing the scope of the function, which is not needed anymore.

In order to track whether or not the instructions we are running are part of a function call, we define a new instruction: `call`. `call` will be used to delimit the invocation of a function in the running program. Instructions stored inside a `call` are part of an active function call. 

In order to store the stack, the state will also be augmented with a new entry, called `stack`, where the local variables are *pushed* (= inserted into) upon function invocation. The stack can either be empty or not:
- an empty stack $\{ \}$ suggests that no functions are active at the present time;
- a non-empty stack stores the local variables of the currently active function in a variable called $h$ (the so-called head of the stack), while a variable called $t$ (the so-called tail of the stack) stores the rest of the stack;

For example $\{ stack := \{ h := \{ x := 1 \}, t := \{ \} \}, x := 1 \}$ would be a state where variable `x` is defined both at the top of the stack and as a global variable.

The new semantics of function invocation will now have to setup the instructions of the function in a way that is recognizable by subsequent evaluations, therefore inside a `call` container instruction, and at the same time setting up the state to contain an extended stack.
    
$\text{eval}(<V(c_1, c_2, \dots)>, S) \rightarrow <call(L)>, S[\text{stack} := \{ h := \{ p_1 := \text{c_1}, p_2 := \text{c_2}, \dots \}, t := S[\text{stack}] \}]$ where $S[V] \rightarrow <(p_1, p_2, \dots) \Rightarrow L>$ and $c_1, \dots$ are all constant values.

Note that we cannot directly assign the arguments $c_1, c_2, ...$ to the parameters $p_1, p_2, ...$ inside the stack in the state, because the arguments could be expressions (for example, 3 + 1) instead of atomic values, so we need to execute the code to evaluate it. This means that, before the rule above can become active, we need to check if some of the arguments are not (yet) constant, in a left-to-right fashion:

$\text{eval}(<V(\dots, E_i, \dots)>, S) \rightarrow V(\dots, E_i', \dots)>, S'$ where $\text{eval}(<E_i>,S) \rightarrow <E_i'>,S'$ and $E_i$ is the first argument from the left which is not a constant.

Let us now consider the very simple program `incr_x := <k => x := x + k>; incr_x(1)` where the initial state is $\{ k := 0, x := 0 \}$. We now expect that, inside the body of the function, `k` will refer to $S[stack][h][k]$ instead of $S[k]$, because local variables must take precedence and also because we do not want to accidentally erase information stored in the global variable because of a local change that only makes sense inside the function itself. 

Implementing this new lookup strategy will therefore require two rules, in order to first look at the head of the stack and then, if nothing is found, in the globals:
- first we look in the stack: $\text{eval}(<V>, S) \rightarrow ( <S[\text{stack}][h][V]>, S )$, where `V` is a variable name and `V` exists in the stack ($\exists S[\text{stack}][h][V]$)
- then we look in the globals, with the rule we already defined in one of the early chapters: $\text{eval}(<V>, S) \rightarrow ( <S[V]>, S)$, where `V` is a variable name which does not exist in the stack ($\nexists S[\text{stack}][h][V]$).

Assigning to variables should also respect the scoping rules. In particular, if the stack is empty, then we will always look a variable up in the global scope. If the variable already exists in the local scope, or if it does not exist locally or globally, then we write it to the local scope. If the variable does not exist in the global scope, then we write it to the local scope.

This means that the semantics of writing to variables is adjusted to the following rules:

- $eval(<V := C>, S) \rightarrow S[stack := S[stack][h := S[stack][h][V := C]]]$ when $\exists S[stack][h][V] \vee \nexists S[V]$
- $eval(<V := C>, S) \rightarrow S[V := C]$ otherwise

<div class="alert alert-block alert-info">
Notice that we have implicitly removed `eval_expr`, and are now using `eval` for both statements and expressions. This has the important side-effect that evaluating expressions can now alter the state, which will be needed when allowing function invocations as part of expressions. For the moment though take notice: all rules that made up `eval_expr` are lifted inside  `eval`, and return an unchanged state.
</div>

Running the `call` instruction simply runs the internal instruction stored inside the `call`, as long as it is not `done`:

$eval(<call(I)>, S) \rightarrow ( <call(I')>, S' )$, where $I$ is an instruction, $I \neq \texttt{<done>}$ and $eval(<I>, S) \rightarrow ( <I'>, S' )$

When a function terminates, then its body becomes `done` and we reach the combination `call(done)`: then we must _pop_ the stack, that is remove the last set of local variables in order for the function to be properly cleaned up:

$eval(<call(done)>, S) \rightarrow <done>, S[stack := S[stack][t]]$

The original example now behaves in a significantly different, and more logical, way: the global variable `k` remains undisturbed throughout the evaluations of the `incr_x` functions:

```
k := 0
x := 0
incr_x := <k => x := x + k>
incr_x(1)
incr_x(3)
...
```

|Program | State|
|:-------:|:-----:|
| `< k := 0 ; ... >` | $\{stack:=\{\} \}$|
| `< x := 0 ; ... >` | $\{ k := 0 , stack:=\{\} \}$|
| `< incr_x := <k => x := x + k> ; ... >` | $\{ k := 0 , x:=0, stack:=\{\} \}$|
| `< incr_x(1) ; ... >` | $\{ k := 0 , x:=0 , incr\_x := <k \Rightarrow x := x + k>,stack:=\{\} \}$|
| `< call(k:=1; x := x + k); … >` | $\{ k := 0 , x:=0 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{h:=\{k:=null\},t:=\{\}\} \}$|
| `< call( x := x + k); … >` | $\{ k := 0 , x:=0 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{h:=\{k:=1\},t:=\{\}\} \}$|
| `< call( x := 0 + k); … >` | $ ... $|
| `< call( x := 0 + 1); … >` | $ ... $|
| `< call( x := 1); … >` | $ ... $|
| `< call( done ); … >` | $\{ k := 0 , x:=1 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{h:=\{k:=1\},t:=\{\}\} \}$|
| `< incr_x(3) ; ... >` | $\{ k := 0 , x:=1 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{\} \}$|
| `< call(k:=3; x := x + k); … >` | $\{ k := 0 , x:=1 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{h:=\{k:=null\},t:=\{\}\} \}$|
| `< call(x := x + k); … >` | $\{ k := 0 , x:=1 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{h:=\{k:=3\},t:=\{\}\} \}$|
| `< call(x := 1 + k); … >` | $ ... $|
| `< call(x := 1 + 3); … >` | $ ... $|
| `< call(x := 4); … >` | $ ... $|
| `< call( done ); … >` | $\{ k := 0 , x:=4 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{h:=\{k:=3\},t:=\{\}\} \}$|
| `< … >` | $\{ k := 0 , x:=4 , incr\_x := <k \Rightarrow x := x + k>,stack:= \{\} \}$|

Let us consider a few more examples with name clashes between variables found in different scopes:

```
k := 0
double := <k => x := k*2>
double(1)
double(3)
...
```
|Program | State|
|:-------:|:-----:|
| `< k := 0 ; ... >` | $\{stack:=\{\} \}$|
| `< double := <k => x := k*2> ; ... >` | $\{ k := 0, stack:=\{\} \}$|
| `< double(1) ; ... >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:=\{\} \}$|
| `< call(k:=1; x := k*2 ); … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{h:=\{k:=null\},t:=\{\}\} \}$|
| `< call( x := k*2 ); … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{h:=\{k:=1\},t:=\{\}\} \}$|
| `< call( x := 1*2 ); … >` | $ ... $|
| `< call( x := 2 ); … >` | $ ... $|
| `< call( done ); … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{h:=\{k:=1,x:=2\},t:=\{\}\} \}$|
| `< double(3); … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{\} \}$|
| `< call(k:=3; x := k*2 ); … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{h:=\{k:=null\},t:=\{\}\} \}$|
| `< call( x := k*2 ); … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{h:=\{k:=3\},t:=\{\}\} \}$|
| `< call( x := 3*2 ); … >` |  $ ... $|
| `< call( x := 6 ); … >` | $ ... $|
| `< call( done ); … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{h:=\{k:=3,x:=6\},t:=\{\}\} \}$|
| `< … >` | $\{ k := 0, double := <k \Rightarrow x := k*2 > , stack:= \{\} \}$|

Notice that, in this case, variable x must go in the stack because it is not initialized in the globals! It happens like this in most programming languages that is why in this course it was decided to implement our metalanguage like this.

## Nesting function calls
Thanks to our notion of scoping, each function call is segregated from the others. This extends to nested function calls: a function may call other functions, and this will produce a stack with more than one scope at the state level, and multiple nested `call` statements in the program. For example, consider the following program in which the function `quadruple` invokes another function, `double`. In this case, while we are executing the `double` function, there are two `call` statements inside each other (the outer one for `quadruple`, and the inner one for `double`), and the state contains a stack with two levels:

```
x := 1
double := <() => x := x + x>
quadruple := <() => double(); double()>
quadruple()
```

|Program | State|
|:-------:|:-----:|
| `< x := 1 ; ... >` | $\{ stack:=\{\} \}$|
| `< double := <() => x := x + x> ; ... >` | $\{ x := 1 , stack:=\{\} \}$|
| `< quadruple := <() => double(); double()> ; ... >` | $\{ x := 1 , double := <() \Rightarrow x := x + x> , stack:=\{\} \}$|
| `< quadruple() ; ... >` | $\{ x := 1 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:=\{\} \}$|
| `< call(double(); double()) ; ... >` | $\{ x := 1 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{h:=\{\},t:=\{\}\} \}$|
| `< call(call(x := x + x); double()) ; ... >` | $\{ x := 1 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
| `< call(call(x := 1 + x); double()) ; ... >` | $ ... $|
| `< call(call(x := 1 + 1); double()) ; ... >` | $ ... $|
| `< call(call(x := 2); double()) ; ... >` | $ ... $|
| `< call(call(done); double()) ; ... >` | $\{ x := 2 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
| `< call(double()) ; ... >` | $\{ x := 2 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{h:=\{\},t:=\{\}\} \}$|
| `< call(call(x := x + x)) ; ... >` | $\{ x := 2 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
| `< call(call(x := 2 + x)) ; ... >` | $ ... $|
| `< call(call(x := 2 + 2)) ; ... >` | $ ... $|
| `< call(call(x := 4)) ; ... >` | $ ... $|
| `< call(call(done)) ; ... >` | $\{ x := 4 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{h:=\{\},t:=\{h:=\{\},t:=\{\}\}\} \}$|
| `< call(done) ; ... >` | $\{ x := 4 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{h:=\{\},t:=\{\}\} \}$|
| `< ... >` | $\{ x := 4 , double := <() \Rightarrow x := x + x> , quadruple := <() \Rightarrow double(); double()> , stack:= \{\} \}$|

Let us consider another example. Drawing of complex figures can itself be decomposed into the drawing of smaller figures in sequence, so we could formulate a $3 \times 3$ square as:

```
s := ""
star := <() => s := s + "*">
blank := <() => s := s + " ">
newline := <() => s := s + "\n">
line := <() => star(); star(); star(); newline() >
square := <() => line(); line(); line() >
square()
```

Moreover, even if (some of) these functions use the same parameters, we still do not get clashes because each function variables are referred to a specific scope:

```
k := 1
x := 1
incr_x := <k => x := x + k>
mult_x := <k => x := x * k>
mult_incr_x := <(k,c) => mult_x(k); incr_x(c)>
mult_incr_x(2,3)
...
```

|Program | State|
|:------:|:-----:|
| `< k:=1 ; ... >` | $\{ stack:=\{\} \}$|
| `< x:=1 ; ... >` | $\{ k:=1 , stack:=\{\} \}$|
| `< incr_x := <k => x := x + k> ; … >` | $\{ k:=1 , x:=1 , stack:=\{\} \}$|
| `< mult_x := <k => x := x * k> ; … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , stack:=\{\} \}$|
| `< mult_incr_x := <(k,c) => mult_x(k); incr_x(c)> ; … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , stack:=\{\} \}$|
| `< mult_incr_x(2,3) ; … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{\} \}$|
| `< call(k:=2;c:=3;mult_x(k); incr_x(c)); … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=null,c:=null\},t:=\{\}\} \}$|
| `< call(c:=3;mult_x(k); incr_x(c)); … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=2,c:=null\},t:=\{\}\} \}$|
| `< call(mult_x(k); incr_x(c)); … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=2,c:=3\},t:=\{\}\} \}$|
| `< call(mult_x(2); incr_x(c)); … >` | $…$|
| `< call(call(k:=2;x :=x*k); incr_x(c)); … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=null\},t:=\{h:=\{k:=2,c:=3\},t:=\{\}\}\} \}$|
| `< call(call(x :=x*k); incr_x(c)); … >` | $\{ k:=1 , x:=1 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=2\},t:=\{h:=\{k:=2,c:=3\},t:=\{\}\}\} \}$|
| `< call(call(x :=1*k); incr_x(c)); … >` | $…$|
| `< call(call(x :=1*2); incr_x(c)); … >` | $…$|
| `< call(call(x :=2); incr_x(c)); … >` | $…$|
| `< call(call(done); incr_x(c)); … >` | $\{ k:=1 , x:=2 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=2\},t:=\{h:=\{k:=2,c:=3\},t:=\{\}\}\} \}$|
| `< call(incr_x(c)); … >` | $\{ k:=1 , x:=2 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=2,c:=3\},t:=\{\}\} \}$|
| `< call(incr_x(3)); … >` | $…$|
| `< call(call(k:=3; x:=x+k)); … >` | $\{ k:=1 , x:=2 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=null\},t:=\{h:=\{k:=2,c:=3\},t:=\{\}\}\} \}$|
| `< call(call(x:=x+k)); … >` | $\{ k:=1 , x:=2 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=3\},t:=\{h:=\{k:=2,c:=3\},t:=\{\}\}\} \}$|
| `< call(call(x:=2+k)); … >` | $…$|
| `< call(call(x:=2+3)); … >` | $…$|
| `< call(call(x:=5)); … >` | $…$|
| `< call(call(done)); … >` | $\{ k:=1 , x:=5 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=3\},t:=\{h:=\{k:=2,c:=3\},t:=\{\}\}\} \}$|
| `< call(done); … >` | $\{ k:=1 , x:=5 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{h:=\{k:=2,c:=3\},t:=\{\}\} \}$|
| `<  … >` | $\{ k:=1 , x:=5 , incr\_x := <k \Rightarrow x := x + k> , mult\_x := <k \Rightarrow x := x * k> , mult\_incr\_x := <(k,c) \Rightarrow mult\_x(k); incr\_x(c)> , stack:=\{\} \}$|

<div class="alert alert-block alert-info">
As a closing note, it is worth mentioning that it is not just a random stroke of luck that allowed us to use the very same semantic rules for $eval$ in order to suddenly support arbitrarily nested function calls.

When we observed the problem of name clashes in the presence of a single function call, we might have made unsafe assumptions. The simplest assumption along these lines would have been that there is only one function called at a given time. This would have been an over-generalization of the example, but it would have been incorrect seen how the body of a function just contains code, so inside the function we can easily invoke other functions!

There is no universal recipe to make sure that every time we define a system we will cover all important cases, but it is important to always be on the lookout for such circumstances: always assume that the *most complex possible* combination of factors will present itself, and reason accordingly. Large scale, concurrent access, and constructs nested inside each other are the daily issues with which a programmer regularly contends!
</div>